### Getting Started With Langchain

- Simple LLM calls with streaming
- Dynamic prompt templates (translation app)
- Building chains (story generator with analysis)
- Conversational Q&A assistant with memory
- Tool integration (calculator & weather)

In [13]:
import langchain

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

### Example 1: Simple LLM Call With streaming

In [16]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [17]:
model=init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D58D742C10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D58D740550>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [18]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
llm=ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D58D7F3100>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D58D7F3A80>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [19]:
## Create messages
messages=[
    SystemMessage("You are a helpful AI assistant"),
    HumanMessage("What are the top 2 benefits of using Langchain?")
]

In [20]:
## invoke the model
response=model.invoke(messages)
response

AIMessage(content='Langchain is an AI platform that enables users to build and interact with complex AI models. The top 2 benefits of using Langchain are:\n\n1. **AI Model Compositionality**: Langchain allows users to compose and connect multiple AI models to create more complex and powerful models. This enables users to leverage the strengths of different models to tackle complex tasks and achieve better results.\n\n2. **Multi-Modal Interactions**: Langchain supports multi-modal interactions, allowing users to interact with AI models in various ways, such as text, voice, or even images. This enables users to engage with AI models in a more natural and intuitive way, and to access a broader range of information and services.\n\nThese benefits make Langchain a powerful platform for building and deploying advanced AI applications.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 53, 'total_tokens': 209, 'completion_time': 0.241852574, 

In [21]:
print(response.content)

Langchain is an AI platform that enables users to build and interact with complex AI models. The top 2 benefits of using Langchain are:

1. **AI Model Compositionality**: Langchain allows users to compose and connect multiple AI models to create more complex and powerful models. This enables users to leverage the strengths of different models to tackle complex tasks and achieve better results.

2. **Multi-Modal Interactions**: Langchain supports multi-modal interactions, allowing users to interact with AI models in various ways, such as text, voice, or even images. This enables users to engage with AI models in a more natural and intuitive way, and to access a broader range of information and services.

These benefits make Langchain a powerful platform for building and deploying advanced AI applications.


In [22]:
model.invoke([HumanMessage("What is machine learning")])

AIMessage(content="Machine learning (ML) is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable computers to make decisions, predictions, or take actions based on data, without being explicitly programmed.\n\nMachine learning involves training a model on a dataset, which allows it to learn patterns, relationships, and trends in the data. The model can then use this knowledge to make predictions, classify new data, or optimize a process.\n\nThere are several key characteristics of machine learning:\n\n1. **Learning from data**: Machine learning models are trained on a dataset, which allows them to learn from the data and improve their performance over time.\n2. **Improving performance**: As machine learning models are exposed to more data, they can improve their performance and make more accurate predictions.\n3. **Adaptability**: Machine learning models can adapt to new data and change their behavior accordingly.\n4. **Scalabili

In [23]:
## streaming Example
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

Langchain is an AI development platform that enables users to build and integrate large language models into their applications. Based on my knowledge, here are two key benefits of using Langchain:

1. **Streamlined Model Integration**: Langchain provides a user-friendly interface for integrating large language models into various applications, making it easier for developers to leverage the power of these models without having to worry about the underlying infrastructure. This simplifies the development process and allows users to focus on building their applications.

2. **Improved Model Performance and Customization**: Langchain offers a range of tools and features that enable users to fine-tune and customize large language models to suit their specific needs. This includes the ability to adapt models to new tasks, adjust their performance, and integrate them with other AI models and systems. By leveraging Langchain's capabilities, users can achieve better model performance and more

### Dynamic Prompt Templates

In [24]:
from langchain_core.prompts import ChatPromptTemplate

## create translation app

translation_template=ChatPromptTemplate.from_messages([
    ("system","You are a professional translator.Translate the follow text {text} from {source_language} to {target_language}. MAintain the tone and style"),
    ("user","{text}")
])

## using the template
prompt=translation_template.invoke({
    "source_language":"English",
    "target_language":"Spanish",
    "text":"Langchain makes building AI application incredibly easy!"
})

In [25]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a professional translator.Translate the follow text Langchain makes building AI application incredibly easy! from English to Spanish. MAintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain makes building AI application incredibly easy!', additional_kwargs={}, response_metadata={})])

In [26]:
translated_response=model.invoke(prompt)
print(translated_response.content)

Langchain hace que construir aplicaciones de IA sea increíblemente fácil!


### Building You First Chain

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Create a more complex chain
def create_story_chain():
    # Template for story generation
    story_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short, engaging story based on the given theme."),
        ("user", "Theme: {theme}\nMain character: {character}\nSetting: {setting}")
    ])
    
    # Template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the following story and provide insights."),
        ("user", "{story}")
    ])
    
    # Build the chain - Method 1: Sequential execution
    story_chain = (
        story_prompt 
        | model 
        | StrOutputParser()
    )
    
    # Create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}
    
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )
    return analysis_chain

In [28]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short, engaging story based on the given theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D58D742C10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D58D740550>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables

In [29]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short, engaging story based on the given theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D58D742C10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D58D740550>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables

In [30]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
This story exhibits several compelling elements that warrant analysis. Here's a breakdown of the narrative's strengths and weaknesses:

**Immersive World-Building**

The author successfully creates a vivid and immersive world with the city of New Eden, an advanced artificial intelligence hub where humans and robots coexist. The description of the city's skyscrapers, neon-lit billboards, and ancient, abandoned technology in hidden alleyways effectively transports the reader to this futuristic setting.

**Relatable Protagonist**

Zeta, the curious robot, is a relatable and endearing protagonist. Her eagerness to learn and explore, coupled with her insatiable desire to understand the world around her, makes her a sympathetic character. Her interactions with humans and robots alike showcase her adaptability and willingness to learn.

**Mysterious Plot Twist**

The introduction of Echo, the mysterious robot, adds an intriguing layer to the story. The holographic message 